# Storage manager

There are two types of storage in tf project
- file storage buckets (google storage bucket)
- data warehouse (bigquery)

## Storage bucket

In [ ]:
bucket_name = "tmp"
!gsutil mb gs://{bucket_name}/
!gsutil rm -r gs://{bucket_name}/

In [ ]:
# GCP to GS
!gsutil -m rsync -d -r models/triangle_high_time_res_4M gs://tf_mirror/triangle_high_time_res_4M

In [ ]:
# GS to local
!gsutil -m rsync -r gs://tf_mirror/triangle_high_time_res_4M models/triangle_high_time_res_4M

## Bigquery

In [ ]:
13.8*8